# 1. Install and Import Dependencies

In [ ]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch #for PyTorch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3. Encode and Calculate Sentiment

In [ ]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')
# Tokenizes the input text and converts it into a tensor suitable for input to the model.

In [ ]:
result = model(tokens)
# Passes the tokenized input to the model to get predictions.

In [ ]:
result.logits
# Retrieves the raw output from the model, which are the logits 

In [ ]:
int(torch.argmax(result.logits))+1
# Finds the index of the maximum logit value, converts it to an integer

# 4. Collect Reviews

In [ ]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [ ]:
reviews

# 5. Load Reviews into DataFrame and Score

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))